In [20]:
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('tools/')

import FM_FTRL_machine

from sklearn.model_selection import KFold

In [19]:
help(tffm)

Help on package tffm:

NAME
    tffm

PACKAGE CONTENTS
    base
    core
    models
    utils

CLASSES
    tffm.base.TFFMBaseModel(sklearn.base.BaseEstimator)
        tffm.models.TFFMClassifier
        tffm.models.TFFMRegressor
    
    class TFFMClassifier(tffm.base.TFFMBaseModel)
     |  Factorization Machine (aka FM).
     |  
     |  This class implements L2-regularized arbitrary order FM model with logistic
     |  loss and gradient-based optimization.
     |  
     |  Only binary classification with 0/1 labels supported.
     |  
     |  See TFFMBaseModel and TFFMCore docs for details about parameters.
     |  
     |  Method resolution order:
     |      TFFMClassifier
     |      tffm.base.TFFMBaseModel
     |      sklearn.base.BaseEstimator
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, **init_params)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  fit(self, X, y, sample_weight=None

In [2]:
trainFile = 'round1_ijcai_18_train_20180301.txt'
testFile = 'round1_ijcai_18_test_a_20180301.txt'
dataDir = '../../Data/advertisment/Raw/'

In [3]:
dfTrain = pd.read_table(dataDir+trainFile,sep=' ')
dfTest = pd.read_table(dataDir+testFile,sep=' ')

In [4]:
print(dfTrain.dtypes)
print(dfTrain.shape)

instance_id                    int64
item_id                        int64
item_category_list            object
item_property_list            object
item_brand_id                  int64
item_city_id                   int64
item_price_level               int64
item_sales_level               int64
item_collected_level           int64
item_pv_level                  int64
user_id                        int64
user_gender_id                 int64
user_age_level                 int64
user_occupation_id             int64
user_star_level                int64
context_id                     int64
context_timestamp              int64
context_page_id                int64
predict_category_property     object
shop_id                        int64
shop_review_num_level          int64
shop_review_positive_rate    float64
shop_star_level                int64
shop_score_service           float64
shop_score_delivery          float64
shop_score_description       float64
is_trade                       int64
d

In [5]:
featInput = []
keyList = ['instance_id','item_id','user_id','context_id','shop_id','is_trade']
for var in dfTrain.columns:
    if not var in keyList and dfTrain[var].dtypes != 'object':
        featInput.append(var)
print(featInput)
print(len(featInput))

['item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'context_timestamp', 'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description']
18


<font color=#0099ff size=5 face="黑体">处理日期</font>

In [6]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return value.tm_hour


In [7]:
dfTrain['context_timestamp_hour'] = dfTrain['context_timestamp'].map(timestamp_datetime)
try: featInput.remove('context_timestamp')
except: print('context_timestamp has been deleted')
if not 'context_timestamp_hour' in featInput:
    featInput.append('context_timestamp_hour')

<font color=#0099ff size=5 face="黑体">One Hot变形</font>

In [8]:
dummyInput = []
for var in featInput:
    if 'int' in str(dfTrain[var].dtypes):
        print(var)
        dfTrain[var].replace({-1:np.nan},inplace=True)
        dfTmp = pd.get_dummies(dfTrain[var],var)
        dfTrain = pd.concat([dfTrain,dfTmp],axis=1)
        try:
            featInput.remove(var)
            dummyInput +=dfTmp.columns.tolist()
        except:
            print('Removed')
featInput+=dummyInput

item_brand_id
item_price_level
item_collected_level
user_gender_id
user_occupation_id
context_page_id
shop_star_level
context_timestamp_hour


In [9]:
print(dfTrain.columns)
print(dfTrain.shape)

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level',
       ...
       'context_timestamp_hour_14', 'context_timestamp_hour_15',
       'context_timestamp_hour_16', 'context_timestamp_hour_17',
       'context_timestamp_hour_18', 'context_timestamp_hour_19',
       'context_timestamp_hour_20', 'context_timestamp_hour_21',
       'context_timestamp_hour_22', 'context_timestamp_hour_23'],
      dtype='object', length=2187)
(478138, 2187)


<font color=#0099ff size=5 face="黑体">CV 创建模型输入</font>

In [10]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
for trainIndex,testIndex in cv.split(dfTrain[keyList]):
    trainX,testX = dfTrain.loc[trainIndex,featInput],dfTrain.loc[testIndex,featInput]
    trainY,testY = dfTrain.loc[trainIndex,'is_trade'],dfTrain.loc[testIndex,'is_trade']
    
    fm_dim = trainX.shape[1]
    fm_initDev = .01
    hashSalt = "salty"

    alpha = .1
    beta = 1.

    alpha_fm = .01
    beta_fm = 1.

    p_D = 15
    D = 2 ** p_D

    L1 = 1.0
    L2 = .1
    L1_fm = 2.0
    L2_fm = 3.0
    
    
    
    
    
    break

In [ ]:
import tensorflow as tf
model = tffm.TFFMClassifier(
    order=6,
    rank=10,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    n_epochs=100,
    batch_size=-1,
    init_std=0.001,
    input_type='dense'
)
model.fit(trainX.values, trainY.values, show_progress=True)

In [12]:
fm_dim = trainX.shape[1]
fm_initDev = .01
hashSalt = "salty"

alpha = .1
beta = 1.

alpha_fm = .01
beta_fm = 1.

p_D = 15
D = 2 ** p_D

L1 = 1.0
L2 = .1
L1_fm = 2.0
L2_fm = 3.0

In [15]:
def logLoss(p, y):
    ''' 
    calculate the log loss cost
    p: prediction [0, 1]
    y: actual value {0, 1}
    '''
    p = max(min(p, 1. - 1e-15), 1e-15)
    return - log(p) if y == 1. else -log(1. - p)

In [24]:
learner = FM_FTRL_machine.FM_FTRL_machine(fm_dim, fm_initDev, L1, L2, L1_fm, L2_fm, D, alpha, beta, alpha_fm = alpha_fm, beta_fm = beta_fm)

print("Start Training:")
n_epochs = 5
for e in range(n_epochs):
    
    # if it is the first epoch, then don't use L1_fm or L2_fm
    if e == 0:
        learner.L1_fm = 0.
        learner.L2_fm = 0.
    else:
        learner.L1_fm = L1_fm
        learner.L2_fm = L2_fm
    cvLoss = 0.
    cvCount = 0.
    progressiveLoss = 0.
    progressiveCount = 0.
    
    for trainIndex in range(0,trainX.shape[0],40000):
        trainXTmp = trainX.iloc[trainIndex:trainIndex+40000]
        trainYTmp = trainY.iloc[trainIndex:trainIndex+40000]
        
        p = learner.predict(trainXTmp)
        loss = logLoss(p, trainYTmp)
        learner.update(trainXTmp, p, trainYTmp)

        progressiveLoss += loss
        progressiveCount += 1.
    print('completed')

Start Training:


TypeError: list indices must be integers or slices, not str